# Módulo 2 Actividad 5

*1. Para nuestros datos de supermercado, crear una lista de los clientes, con el gasto hecho en compras de cada uno. Si no ha comprado, debe aparecer con un 0*

In [61]:
import pandas as pd

detalle = pd.read_csv('detalle.csv')
boletas = pd.read_csv('boletas.csv')
clientes = pd.read_csv('clientes.csv', encoding='latin1')
productos = pd.read_csv('productos.csv', encoding='latin1')

df_detalle = pd.DataFrame(detalle)
df_boletas = pd.DataFrame(boletas)
df_clientes = pd.DataFrame(clientes)
df_productos = pd.DataFrame(productos)

df_clientes

,rut,nombre,direccion,comuna,fecha_nacimiento,telefono,email
0,11223344-4,Tomás Ramírez,NaN,Arica,1994-12-31,NaN,tomas@example.com
1,11223344-5,Laura Rojas,Calle Virtual 567,Antofagasta,1998-07-12,945678321.0,laura@example.com
2,12345678-9,Juan Perez,Calle Falsa 123,Santiago,2000-05-15,912345678.0,juan@example.com
3,23456789-1,Pedro Rodríguez,NaN,Valparaíso,1988-03-10,912345678.0,pedro@example.com
4,33445566-5,Camila Torres,Calle Cibernética 456,NaN,1993-02-14,977894512.0,camila@example.com
5,33445566-7,Mateo Diaz,NaN,Viña del Mar,2002-02-28,967894512.0,mateo@example.com
6,34567890-2,Carla González,Calle Inventada 789,NaN,1999-12-05,934567890.0,carla@example.com
7,45678901-3,Diego Martínez,Avenida Irreal 987,La Serena,NaN,NaN,diego@example.com
8,55667788-6,Matias Castro,Avenida Innovación 321,NaN,1992-06-08,923478965.0,NaN
9,55667788-9,Valentina Herrera,Avenida Tecnológica 123,NaN,1997-04-05,923478965.0,valentina@example.com


In [48]:
df_detalle_con_precios = pd.merge(df_detalle, df_productos, left_on='codigo_producto', right_on='codigo', how='left')
df_detalle_con_precios.drop('codigo_producto', axis=1, inplace=True)
df_detalle_con_precios['Total'] = df_detalle_con_precios['cantidad'] * df_detalle_con_precios['precio']

total_boleta = df_detalle_con_precios.groupby('numero_boleta')['Total'].sum().reset_index()
total_boleta.rename(columns={'Total': 'total_boleta'}, inplace=True)
df_boletas_con_total = pd.merge(df_boletas, total_boleta, left_on='numero', right_on='numero_boleta', how='left')
df_boletas_con_total.drop('numero_boleta', axis=1, inplace=True)

total_por_cliente = df_boletas_con_total.groupby('rut_cliente')['total_boleta'].sum().reset_index()
total_por_cliente.rename(columns={'total_boleta': 'Total por Cliente'}, inplace=True)
df_clientes_con_total = pd.merge(df_clientes, total_por_cliente, left_on='rut', right_on='rut_cliente', how='left')
df_clientes_con_total.drop(['rut', 'direccion', 'comuna', 'fecha_nacimiento', 'telefono', 'email', 'rut_cliente'], axis=1, inplace=True)
df_clientes_con_total.fillna(0, inplace=True)

df_clientes_con_total


,nombre,Total por Cliente
0,Tomás Ramírez,0.0
1,Laura Rojas,98500.0
2,Juan Perez,133500.0
3,Pedro Rodríguez,286200.0
4,Camila Torres,0.0
5,Mateo Diaz,35000.0
6,Carla González,14000.0
7,Diego Martínez,3600.0
8,Matias Castro,0.0
9,Valentina Herrera,121900.0


*2. Crea un DataFrame con datos de ventas por producto y región. Utiliza multi-índices para representar la información por año y trimestre. Luego, calcula la suma de ventas por trimestre para cada producto.*

In [63]:
# Crear DataFrame de productos y cambiar el nombre de la columna 'nombre' a 'producto'
df_productos.rename(columns={'nombre': 'producto'}, inplace=True)
df_clientes.fillna('Sin Informacion', inplace=True)

# Convertir la columna 'fecha' de las boletas a tipo datetime
df_boletas['fecha'] = pd.to_datetime(df_boletas['fecha'])

# Unir los DataFrames para obtener la información completa de las ventas
df_unido = pd.merge(df_boletas, df_detalle, left_on='numero', right_on='numero_boleta')
df_unido = pd.merge(df_unido, df_productos, left_on='codigo_producto', right_on='codigo')
df_unido = pd.merge(df_unido, df_clientes, left_on='rut_cliente', right_on='rut')

# Extraer el año y el trimestre de la fecha de la boleta
df_unido['year'] = df_unido['fecha'].dt.year
df_unido['quarter'] = df_unido['fecha'].dt.quarter

ventas_por_trimestre = df_unido.pivot_table(index=['producto', 'comuna'], 
                                            columns=['year', 'quarter'], 
                                            values='cantidad', 
                                            aggfunc='sum', 
                                            fill_value=0)

df_ventas_por_trimestre = pd.DataFrame(ventas_por_trimestre)

df_ventas_por_trimestre

'''Debo mejorar el código para que solo fillna() a la columna comuna'''

# Crear multi-índices para representar la información por año, trimestre, producto y región
#df_unido.set_index(['year', 'quarter', 'producto', 'comuna'], inplace=True)

# Calcular la suma de ventas por trimestre para cada producto
#ventas_por_trimestre = df_unido.groupby(['year', 'quarter', 'comuna', 'producto']).agg({'cantidad': 'sum'})

#ventas_por_trimestre





C:\Users\fjib2\AppData\Local\Temp\ipykernel_11692\3106486780.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Sin Informacion' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_clientes.fillna('Sin Informacion', inplace=True)


year                             2023
quarter                             4
producto         comuna              
Aceite de cocina Santiago           6
                 Sin Informacion    5
Agua mineral     Iquique            3
                 La Serena          3
Arroz            Santiago           9
                 Sin Informacion   15
Azúcar           Iquique            7
                 Puerto Montt       3
                 Santiago           2
Café             Antofagasta        4
                 Sin Informacion    1
                 Valparaíso         8
Carne de pollo   Sin Informacion    6
Fideos           Antofagasta        5
                 Concepción         5
                 Sin Informacion    2
Frutas frescas   Sin Informacion    2
Galletas         Antofagasta        7
Harina           Antofagasta        1
                 Concepción         1
                 Iquique            3
                 Santiago           2
                 Sin Informacion    2
Helado           Antofagasta        4
                 Temuco             4
                 Viña del Mar       2
Leche            Puerto Montt       1
                 Santiago           1
Mantequilla      Valparaíso         4
Pescado          Antofagasta        4
                 Rancagua           9
                 Valparaíso        20
Queso            Viña del Mar       5
Verduras         Iquique            1
                 Rancagua           5
                 Valparaíso        10
Yogur            Sin Informacion   15